### Метод Якобі

In [1]:
import numpy as np
from math import sqrt
import sys

In [2]:
# дані для тестування
def test_1():
    matrix = [[1., 1., 3.], [1., 5., 1.], [3., 1., 1.]]
    return np.array(matrix)

def test_2():
    matrix = [[1.6, 0.7, 0.8], [0.7, 1.6, 0.3], [0.8, 0.3, 1.6]]
    return np.array(matrix)

def test_3():
    matrix = [[5., 1., -3.], [3., 0., -2.], [-4., -1., 1.]]
    return np.array(matrix)

def test_4():
    matrix = [[1., 0., 0., 0.], [0., 0., 0., 0.], [1., 0., 0., 0.], [0., 0., 0., 1.]]
    return np.array(matrix)


In [ ]:
# допоміжні функції
def find_max_element(matrix):
    max_el = 0.
    max_indx = (0, 0)
    for index, el in np.ndenumerate(matrix):
        if ((index[0] != index[1])):
            if (max_el < abs(el)):
                max_el = abs(el)
                max_indx = index
    return max_el, max_indx

def create_matrix_T(matrix, index): # index is tuple (i, j)
    matrix_T = np.zeros_like(matrix, dtype=float)
    np.fill_diagonal(matrix_T, 1)
    p = 2 * matrix[index]
    q = matrix[index[1]][index[1]] - matrix[index[0]][index[0]]
    #print(q)
    if (q == 0):
        c = sqrt(2) / 2
        s = sqrt(2) / 2
    else:
        d = sqrt(p ** 2 + q ** 2)
        r = abs(q) / (2 * d)
        c = sqrt(0.5 + r)
        if (p * q >= 0):
            s = sqrt(0.5 - r)
        else:
            s = -sqrt(0.5 - r)
    matrix_T[index[0]][index[0]] = c
    matrix_T[index[1]][index[1]] = c
    matrix_T[index[0]][index[1]] = s
    matrix_T[index[1]][index[0]] = -s

    return matrix_T

def create_matrix_B(matrix, matrix_T, index):
    i = index[0]
    j = index[1]
    matrix_B = matrix.copy()
    # new diagonal elements
    matrix_B[i][i] = (matrix_T[i][i] ** 2) * matrix[i][i] + (matrix_T[i][j] ** 2) * matrix[j][j] - 2 * matrix_T[i][i] * matrix_T[i][j] * matrix[i][j]
    matrix_B[j][j] = (matrix_T[i][j] ** 2) * matrix[i][i] + (matrix_T[i][i] ** 2) * matrix[j][j] + 2 * matrix_T[i][i] * matrix_T[i][j] * matrix[i][j]
    # other elements of B
    matrix_B[i][j] = 0.
    matrix_B[j][i] = 0.

    for m in range(matrix.shape[0]):
        if ((m != i) & (m != j)):
            matrix_B[i][m] = matrix_T[i][i] * matrix[m][i] - matrix_T[i][j] * matrix[m][j]
            matrix_B[m][i] = matrix_T[i][i] * matrix[m][i] - matrix_T[i][j] * matrix[m][j]
            matrix_B[j][m] = matrix_T[i][j] * matrix[m][i] + matrix_T[i][i] * matrix[m][j]
            matrix_B[m][j] = matrix_T[i][j] * matrix[m][i] + matrix_T[i][i] * matrix[m][j]
    return matrix_B

def check_stop_condition(matrix_B, eps):
    for index, el in np.ndenumerate(matrix_B):
        if (index[0] != index[1]):
            if (abs(el) > eps):
                return False
    return True

def get_trace_matrix(matrix):
    trace = 0
    for index, el in np.ndenumerate(matrix):
        if (index[0] == index[1]):
            trace += el
    return trace

In [ ]:
def Jaccobi_method(matrix, eps):
    list_Ti = []
    sequence_max = []
    sequence_traces = []
    k = 0
    while True:
        #print(matrix)
        max_el, index = find_max_element(matrix)
        #print(max_el, index)
        sequence_max.append(max_el)
        matrix_T = create_matrix_T(matrix, index)
        #print(matrix_T)
        matrix_B = create_matrix_B(matrix, matrix_T, index)
        trace = get_trace_matrix(matrix_B)
        sequence_traces.append(trace)
        #print(matrix_B)
        if not (check_stop_condition(matrix_B, eps)):
            matrix = matrix_B.copy()
            list_Ti.append(matrix_T)
        else:
            list_Ti.append(matrix_T)
            break
    return matrix_B, list_Ti, sequence_max, sequence_traces

def find_vectors(list_Ti):
    product = np.zeros_like(list_Ti[0], dtype=float)
    np.fill_diagonal(product, 1)
    for matrix in list_Ti:
        product = product.dot(matrix)
    return product


In [ ]:
# виведення для методу Якобі
EPS = sys.float_info.epsilon

A_1 = test_1()
val_matrix_1, T_list_1, seq_max_1, seq_trace_1 = Jaccobi_method(A_1.copy(), EPS)
Q_1 = find_vectors(T_list_1)

print('Initial matrix \n {}'.format(A_1))
print('A matrix with values on a diagonal \n {}'.format(val_matrix_1))
print('A matrix with vectors \n {}'.format(Q_1))

print('Check')
Q_1T = np.transpose(Q_1)
print(Q_1T.dot(Q_1))
print('\n')
x = np.matmul(Q_1, val_matrix_1)
print(x.dot(Q_1T) - A_1)

print('Sequence of max element\'s of absolute values \n {}'.format(seq_max_1))
print('Sequence of the matrix\' traces \n {}'.format(seq_trace_1))

### QR розклад

In [3]:
# допоміжні функції
def sign_plus(x):
    if x >= 0:
        return 1
    return -1

In [4]:
# Обчислення матриці Хаусхолдера
def create_H(matrix, i, j):
    E = np.zeros_like(matrix, dtype=float)
    np.fill_diagonal(E, 1)
    summ = 0
    for k in range(i, matrix.shape[1]):
        summ += matrix[k][j] ** 2
    beta = sign_plus((-1)* matrix[i][j]) * sqrt(summ)
    if ((2 * (beta ** 2) - 2 * matrix[i][j] * beta) == 0):
        return E # тобто пропускаємо крок, оскільки вже є потрібні нулі в матриці
    else:
        mu = 1 / sqrt(2 * (beta ** 2) - 2 * matrix[i][j] * beta)
    vector_w = np.zeros(matrix.shape[1])
    for k in range(matrix.shape[1]):
        if (k < i):
            vector_w[k] = 0
        elif (k == i):
            vector_w[k] = mu * (matrix[i][j] - beta)
        else:
            vector_w[k] = mu * matrix[k][j]
    vector_w_T = vector_w.reshape(1, vector_w.size)
    vector_w = vector_w.reshape(vector_w.size, 1)
    H = E - 2 * vector_w.dot(vector_w_T)
    return H

In [5]:
# Матриця Хесенберга
def create_Hesenberg_matrix(matrix):
    list_Hi = []
    for j in range(matrix.shape[1] - 2):
        for i in range(j+1, matrix.shape[0]):
            if ((i - j) == 1):
                H = create_H(matrix, i, j)
                list_Hi.append(H)
                matrix = H.dot(matrix)
    return matrix, list_Hi

In [6]:
# Створення подібної до початкової
def create_similar_B0(Hesenberg_matrix, list_Hi):
    product_H = np.zeros_like(list_Hi[0], dtype=float)
    np.fill_diagonal(product_H, 1)
    for matrix in list_Hi:
        product_H = product_H.dot(matrix)
    return Hesenberg_matrix.dot(product_H)


In [7]:
# створення матриці Гівенса
def create_G(matrix, i, j, eps=0.000001):  # матриця повороту для QR розкладу
    G = np.zeros_like(matrix, dtype ='float')
    np.fill_diagonal(G, 1)
    if (abs(matrix[i-1][j]) > eps):
        t = matrix[i][j] / matrix[i-1][j]
        c = 1 / sqrt(1 + t ** 2)
        s = t / sqrt(1 + t ** 2)
    else:
        s = 0
        c = 1
    G[i-1][j] = c
    G[i][j + 1] = c
    G[i][j] = -s
    G[i - 1][j + 1] = s
    return G

In [8]:
# QR-розклад
def check_stop_condition_Hivens(matrix_B, eps):
    for index, el in np.ndenumerate(matrix_B):
        if ((index[0] - index[1]) == 1):
            if (abs(el) > eps):
                return False
    return True

def create_under_diagonal(B0):
    list_Gi = []
    R = B0.copy()
    for j in range(B0.shape[1] - 1):
        i = j + 1
        G = create_G(R, i,j)
        R = G.dot(R)
        list_Gi.append(G)
    return R, list_Gi

def create_Q(list_Gi):
    Q = np.zeros_like(list_Gi[0], dtype ='float')
    np.fill_diagonal(Q, 1)
    for G in list_Gi:
        G_T = np.transpose(G)
        Q = Q.dot(G_T)
    return Q

def create_new_B(R, Q, eps):
    B_new = R.dot(Q)
    while (check_stop_condition_Hivens(B_new, eps) != True):
        R, list_Gi = create_under_diagonal(B_new)
        Q = create_Q(list_Gi)
        B_new = R.dot(Q)
    return B_new

In [20]:
# виведення для QR-розкладу
EPS_QR = 0.0001 # точність, з якою зведеться матриця А до трикутної з власними числами на діагоналі
A_2 = test_4()
Hessenberg_matrix, l_H = create_Hesenberg_matrix(A_2.copy())
sim_B = create_similar_B0(Hessenberg_matrix, l_H)
R, l_G = create_under_diagonal(sim_B.copy())
Q = create_Q(l_G)
finall_B = create_new_B(R, Q, EPS_QR)

print('The initial matrix \n {}'.format(A_2))
print('\n')
print('The similar matrix to A in Hessenberg\'s form \n {}'.format(sim_B))
print('\n')
print('A matrix with eigenvalues on a diagonal \n {}'.format(finall_B))
print('\n')
print('The QR factorization of the initial matrix')
print('R: {}'.format(R))
print('Q: {}'.format(Q))
print('The QR factorization of the initial matrix with library')
print('R: {}'.format(np.linalg.qr(sim_B)[1]))
print('Q: {}'.format(np.linalg.qr(sim_B)[0]))

The initial matrix 
 [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


The similar matrix to A in Hessenberg's form 
 [[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.22044605e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]


A matrix with eigenvalues on a diagonal 
 [[ 1.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.25176795e-17  3.25176795e-17  0.00000000e+00  0.00000000e+00]
 [ 1.57009246e-16 -1.57009246e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


The QR factorization of the initial matrix
R: [[ 1.41421356e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.59869434e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.22044605e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.0